In [1]:
## Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as utils
import torch.optim as optim
from logger import Logger

In [2]:
## getting the data as panda frame and then converting to the corrosponding numpy matrix

location='default-of-credit-card-clients.csv'
df=pd.read_csv(location,header=None)

df=df[2:]
print(type(df))
df_num=(df.astype(float)).values
df_num=df_num[:,1:]
print(type(df_num))
print(df_num.shape)

def to_np(x):
    return x.data.cpu().numpy()

<class 'pandas.core.frame.DataFrame'>
<type 'numpy.ndarray'>
(30000, 24)


In [3]:
## Data forming using tensors

df_num.shape
print(df_num[0])
data=df_num[:,:23]
labels=df_num[:,23:]
data=torch.from_numpy(data)
labels=torch.from_numpy(labels)
print(type(data))
print(type(labels))
data=data.float()
labels=labels.float()
print(type(data))
print(type(labels))
raw_input()
print(data.size())
print(labels.size())
raw_input()
## Hyperparameters
batchsize=4
nb_epochs=1
learning_rate=0.0001
momentum=0.01
print("SIZE of labels: ",labels.size())
## [IMPORTANT for forming the dataset]

## the condition is that the dataset should just be a torch tensor, however while raining the input to the model
## should be a Variable torch tensor
dataset=utils.TensorDataset(data,labels)
dataloader=utils.DataLoader(dataset=dataset,batch_size=batchsize)
print('No of batches:',len(dataloader),' of batch size: ',batchsize)
print('DONE')

[  2.00000000e+04   2.00000000e+00   2.00000000e+00   1.00000000e+00
   2.40000000e+01   2.00000000e+00   2.00000000e+00  -1.00000000e+00
  -1.00000000e+00  -2.00000000e+00  -2.00000000e+00   3.91300000e+03
   3.10200000e+03   6.89000000e+02   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   6.89000000e+02   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]
<class 'torch.DoubleTensor'>
<class 'torch.DoubleTensor'>
<class 'torch.FloatTensor'>
<class 'torch.FloatTensor'>

torch.Size([30000, 23])
torch.Size([30000, 1])

('SIZE of labels: ', torch.Size([30000, 1]))
('No of batches:', 7500, ' of batch size: ', 4)
DONE


In [4]:
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet,self).__init__()
        self.fc1_1=nn.Linear(23,12)
        self.fc1_2=nn.Linear(12,6)
        self.relu=nn.ReLU()
        self.fc1=nn.Linear(23,6)
        self.fc2=nn.Linear(6,3)
        self.fc3=nn.Linear(3,1)
        self.sigmoid=nn.Sigmoid()
    def forward(self,x):
        out=self.fc1_1(x)
        out=self.relu(out)
        out=self.fc1_2(out)
        out=self.relu(out)
        out_1=self.fc1(x)
        out=out+out_1
        out=self.relu(self.fc2(out))
        out=self.sigmoid(self.fc3(out))
        return out
net=TestNet()
loss_criteria=nn.BCELoss()
optimizer=optim.Adam(params=net.parameters(),lr=0.001,weight_decay=0.0001)
logger=Logger('./logs')

In [5]:
num_steps=50000
iter_per_epoch=len(dataloader)
data_iter=iter(dataloader)

for i in range(num_steps):
    ## reset the data_iter 
    if((i+1)%iter_per_epoch==0):
        data_iter=iter(dataloader)
    
    train_x,train_y=next(data_iter)
    train_x=Variable(train_x)
    train_y=Variable(train_y)   ## HACKY FIX as train_y already becomes a Variable
        
    ## creating the architecture , this contains a list of all the parameters
    ## loss function


    ## forward propogate
    out=net(train_x)
    ## get the loss
    loss=loss_criteria(out,train_y)
    ## zero the grad
    optimizer.zero_grad()
    ## backprop
    loss.backward()
    ## update the paramters
    optimizer.step()
    
    if((i+1)%100==0):
        print("iter: ",i+1," loss: ",loss.data[0])

    info={
        'loss':loss.data[0]
    }
    for tag,val in info.items():
        logger.scalar_summary(tag,val,i+1)
        
#     for tag, value in net.named_parameters():
#         tag = tag.replace('.', '/')
#         logger.histo_summary(tag, to_np(value),i)
#         logger.histo_summary(tag+'/grad', to_np(value.grad),i)

    

('iter: ', 100, ' loss: ', 0.6165459156036377)
('iter: ', 200, ' loss: ', 0.5793635845184326)
('iter: ', 300, ' loss: ', 0.6270204186439514)
('iter: ', 400, ' loss: ', 0.6160985827445984)
('iter: ', 500, ' loss: ', 0.49757567048072815)
('iter: ', 600, ' loss: ', 0.47514617443084717)
('iter: ', 700, ' loss: ', 0.45939192175865173)
('iter: ', 800, ' loss: ', 0.5880136489868164)
('iter: ', 900, ' loss: ', 0.5841240286827087)
('iter: ', 1000, ' loss: ', 0.9206920862197876)
('iter: ', 1100, ' loss: ', 0.39546939730644226)
('iter: ', 1200, ' loss: ', 0.38110196590423584)
('iter: ', 1300, ' loss: ', 0.37230220437049866)
('iter: ', 1400, ' loss: ', 0.5686999559402466)
('iter: ', 1500, ' loss: ', 0.3464188873767853)
('iter: ', 1600, ' loss: ', 0.5663537979125977)
('iter: ', 1700, ' loss: ', 0.5653536319732666)
('iter: ', 1800, ' loss: ', 0.3298242390155792)
('iter: ', 1900, ' loss: ', 0.5643375515937805)
('iter: ', 2000, ' loss: ', 0.5638386607170105)
('iter: ', 2100, ' loss: ', 0.3160621523857

KeyboardInterrupt: 